In [34]:
from pymongo import MongoClient
import pandas as pd

In [35]:
x = open('mongo_access.csv', 'r').readline().split(",")
ip_address, port, sample_size = (x[0], int(x[1]), int(x[2]))

In [36]:
mongo_client = MongoClient(ip_address, port)
database_reference = mongo_client.twitter
collection_reference = database_reference.tweets

In [37]:
cursor_sampl = collection_reference.aggregate([{'$sample': {'size': 10}}])
twt_df = pd.DataFrame(list(cursor_sampl))
print("Number of Tweets: {:,}".format(len(twt_df)))

Number of Tweets: 10


In [38]:
twt_users = list(twt_df['user'])
twt_users_df = pd.DataFrame(twt_users)
print(twt_users_df.dtypes)
twt_users_df.head()

contributors_enabled                     bool
created_at                             object
default_profile                          bool
default_profile_image                    bool
description                            object
favourites_count                        int64
follow_request_sent                    object
followers_count                         int64
following                              object
friends_count                           int64
geo_enabled                              bool
id                                      int64
id_str                                 object
is_translator                            bool
lang                                   object
listed_count                            int64
location                               object
name                                   object
notifications                          object
profile_background_color               object
profile_background_image_url           object
profile_background_image_url_https

,contributors_enabled,created_at,default_profile,default_profile_image,description,favourites_count,follow_request_sent,followers_count,following,friends_count,...,profile_text_color,profile_use_background_image,protected,screen_name,statuses_count,time_zone,translator_type,url,utc_offset,verified
0,False,Wed Jul 27 21:35:16 +0000 2016,True,False,None,0,None,26,None,0,...,333333,True,False,test5geo1798,503920,Pacific Time (US & Canada),none,None,-28800.0,False
1,False,Fri Sep 08 16:49:07 +0000 2017,True,False,😍😘,714,None,18,None,115,...,333333,True,False,annabekaeska,158,None,none,None,NaN,False
2,False,Sun Jan 22 02:46:40 +0000 2017,True,False,"Father, husband, son to three moms. Grew up pr...",5693,None,1262,None,1469,...,333333,True,False,AllenWeiser58,20766,None,none,None,NaN,False
3,False,Sat Jul 04 23:10:24 +0000 2009,True,False,6'6| Those eyes seem to lie and they're tellin...,80199,None,1814,None,1197,...,333333,True,False,Embrace_Me_Now,109857,Eastern Time (US & Canada),none,None,-18000.0,False
4,False,Wed Feb 18 20:14:45 +0000 2009,False,False,"""Kindness is a language that the blind can see...",481,None,712,None,1617,...,333333,False,False,TheSiteRep,3973,Pacific Time (US & Canada),none,None,-28800.0,False


In [39]:
from scipy.stats import skew

In [40]:
skew(twt_users_df)

/opt/conda/lib/python3.6/site-packages/scipy/stats/stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


TypeError: unsupported operand type(s) for +: 'NoneType' and 'NoneType'

In [41]:
# let's remove the object types (these ones are strings... maybe we can feature engineer them later)
twt_users_df_no_strings = twt_users_df.select_dtypes(exclude=['object'])
twt_users_df_no_strings.dtypes

contributors_enabled               bool
default_profile                    bool
default_profile_image              bool
favourites_count                  int64
followers_count                   int64
friends_count                     int64
geo_enabled                        bool
id                                int64
is_translator                      bool
listed_count                      int64
profile_background_tile            bool
profile_use_background_image       bool
protected                          bool
statuses_count                    int64
utc_offset                      float64
verified                           bool
dtype: object

In [42]:
# try skew again
skew(twt_users_df)

/opt/conda/lib/python3.6/site-packages/scipy/stats/stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


TypeError: unsupported operand type(s) for +: 'float' and 'NoneType'

In [43]:
twt_users_df_no_strings

,contributors_enabled,default_profile,default_profile_image,favourites_count,followers_count,friends_count,geo_enabled,id,is_translator,listed_count,profile_background_tile,profile_use_background_image,protected,statuses_count,utc_offset,verified
0,False,True,False,0,26,0,True,758415480881164288,False,24,False,True,False,503920,-28800.0,False
1,False,True,False,714,18,115,True,906197718246322176,False,0,False,True,False,158,NaN,False
2,False,True,False,5693,1262,1469,True,822998890433507328,False,3,False,True,False,20766,NaN,False
3,False,True,False,80199,1814,1197,True,53784365,False,41,False,True,False,109857,-18000.0,False
4,False,False,False,481,712,1617,True,21235818,False,35,False,False,False,3973,-28800.0,False
5,False,False,False,11319,747,228,True,2178652760,False,20,False,False,False,31229,-28800.0,False
6,False,False,False,511,13084,12099,True,104688573,False,456,True,False,False,39798,-28800.0,False
7,False,False,False,11901,2669,751,True,22561831,False,36,True,True,False,14749,-21600.0,False
8,False,False,False,873,2450,292,True,41771392,False,37,True,True,False,58132,-28800.0,False
9,False,False,False,7173,453,288,True,116536727,False,5,True,True,False,18085,-28800.0,False


In [53]:
# aha! there are NaN values... let's be lazy like the professor and just drop them
twt_users_df_no_strings = twt_users_df_no_strings.dropna()
skew(twt_users_df_no_strings)

TypeError: Cannot cast array data from dtype('O') to dtype('float64') according to the rule 'safe'

In [56]:
bool_cols = ['contributors_enabled', 'default_profile', 'default_profile_image', 'geo_enabled', 
             'is_translator', 'profile_background_tile', 'profile_use_background_image', 'protected', 'verified']
twt_users_df_no_strings[bool_cols] = (twt_users_df_no_strings[bool_cols] == True).astype(int)

In [59]:
twt_users_df_no_strings

,contributors_enabled,default_profile,default_profile_image,favourites_count,followers_count,friends_count,geo_enabled,id,is_translator,listed_count,profile_background_tile,profile_use_background_image,protected,statuses_count,utc_offset,verified
0,0,1,0,0,26,0,1,758415480881164288,0,24,0,1,0,503920,-28800.0,0
3,0,1,0,80199,1814,1197,1,53784365,0,41,0,1,0,109857,-18000.0,0
4,0,0,0,481,712,1617,1,21235818,0,35,0,0,0,3973,-28800.0,0
5,0,0,0,11319,747,228,1,2178652760,0,20,0,0,0,31229,-28800.0,0
6,0,0,0,511,13084,12099,1,104688573,0,456,1,0,0,39798,-28800.0,0
7,0,0,0,11901,2669,751,1,22561831,0,36,1,1,0,14749,-21600.0,0
8,0,0,0,873,2450,292,1,41771392,0,37,1,1,0,58132,-28800.0,0
9,0,0,0,7173,453,288,1,116536727,0,5,1,1,0,18085,-28800.0,0
